# S1-S2 classification

Original file name: **Record.ipynb**

Author: Zhuoran Feng

First committed: 22/06/2020

The notebook reads in raw data, uses Joey's peak finder to distinguish S1 and S2 peaks, and records them into a pickle file.

In [1]:
import Zip
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import time
import math
import pickle

# Channels in the detector
det_ch = { 'tpc': list(range(0, 247+1)), 'veto': list(range(248, 253+1)), 
          'sum_wv': [254], 'busy_on': [255], 'busy_off': [256],
          'hev_on': [257], 'hev_off': [258], 'muon_veto_trigger': [259], } 
n_channels = 260

# The data files
NG = Zip.ReadZipped("/data/nikhefproject/data/xenon1t/NG") # Neutron data
Rn220 = Zip.ReadZipped("/data/nikhefproject/data/xenon1t/Rn220") # Rn-220 data

In [2]:
# From Joey

def Peaks(pulse, printall):
    
    # The standard variables
    q = 0
    left_t, right_t = 0, 0
    baseline = 16382.
    std = 2.3
    ch = pulse.channel
    wave = pulse.raw_data
    
    # The index for the minimum
    minindex = len(wave)
    
    # Variables for the average, the ratio and the slope
    avfound = 0
    samplelist = [wave[0]]
    ratio = 1
    slope = 0
    sw = 0

    s = np.std(wave)
    delta = baseline - np.min(wave)
    if min(s, delta) > 5*std:
        for samples in enumerate(wave):
            base_sub = samples[1] - baseline
            if abs(base_sub) > 5*std:
                q = q + base_sub
            
            # Compute the average of the previous samples
            av = np.average(samplelist)
            
            # left_t is assigned when a significant deviation from the average is recorded
            if abs(samples[1] - av) > 20 and left_t == 0:
                left_t = samples[0]
                
                # The sample value at left_t is kept for the slope calculation
                leftsample = samplelist[-1]
                if printall:
                    print("left_t here")
                avfound = av
             
            # Find the index for the minimum
            if samples[1] == np.min(wave):
                minindex = samples[0]

            # right_t is assigned in a similar way as left_t
            if abs(samples[1] - avfound) < 20 and samples[0] > minindex and right_t == 0: 
                right_t = samples[0]
                if printall:
                    print("right_t here")
                
            # Add the current sample to the sample list
            samplelist.append(samples[1])
            
            # A printing section in case the raw data needs to be analysed
            if printall:
                print(samples)
                print("Average:", av)
                if left_t == 0:
                    print("Average difference:", abs(samples[1] - av))
                else:
                    print("Average difference:", abs(samples[1] - avfound))
                print()
        
        # Calculate the ratio and the slope
        ratio = np.min(wave)/int(avfound)
        delta_x = minindex - left_t
        if delta_x != 0:
            slope = (np.min(wave) - leftsample)/delta_x
        
    else:
        baseline = np.average(wave)
        std = s
    width = right_t - left_t
    if width != 0:
        sw = slope/width
        if -q/width < 3*std or ratio >= 0.99:
            q = 0
    return (ch, q, width, pulse.left+left_t, sw)

## Some small tests

In [4]:
# peak finder test
event = NG.get_single_event(104)


for p in event.pulses:
    (ch, q, width, t0, sw) = Peaks(p, False)
    if q < 0 and sw != 0:
        sw = -sw
        print(ch, ": ", q, width, t0, "| ", sw)

153 :  -92581.0 27 100050 |  5.98941798942
105 :  -90150.0 23 100050 |  1.51304347826
11 :  -45133.0 19 100050 |  15.3684210526
112 :  -84573.0 11 100050 |  20.9090909091
164 :  -50831.0 23 100051 |  5.46956521739
165 :  -55266.0 27 100051 |  3.33862433862
13 :  -46121.0 17 100051 |  2.47058823529
95 :  -49849.0 22 100052 |  3.14772727273
141 :  -54073.0 28 100051 |  8.44047619048
151 :  -54081.0 27 100051 |  2.90476190476
76 :  -47396.0 21 100051 |  7.06349206349
204 :  -56210.0 25 100051 |  12.184
180 :  -61451.0 32 100051 |  7.30729166667
181 :  -57360.0 26 100051 |  15.8974358974
189 :  -60486.0 23 100051 |  5.72173913043
177 :  -55290.0 27 100051 |  7.85185185185
42 :  -48929.0 20 100051 |  21.45
71 :  -47242.0 19 100051 |  5.60526315789
96 :  -50425.0 26 100051 |  23.2692307692
110 :  -48204.0 22 100051 |  3.73636363636
122 :  -49999.0 23 100051 |  7.65217391304
46 :  -49423.0 20 100051 |  1.94285714286
159 :  -51959.0 25 100052 |  5.424
149 :  -79572.0 22 100052 |  5.88636363636

112 :  -257471.0 434 138042 |  0.00104978681252
84 :  -40163.0 4 138137 |  50.25
132 :  -44383.0 7 138213 |  48.8571428571
151 :  -69808.0 81 138261 |  0.0357543690877
95 :  -141753.0 77 138403 |  0.047736047736
74 :  -62094.0 51 138466 |  0.146840958606
16 :  -133554.0 87 138468 |  0.0228516694034
96 :  -273267.0 122 138579 |  0.0193552830968
127 :  -40241.0 4 138650 |  51.5
140 :  -40120.0 4 138704 |  43.5
72 :  -125293.0 128 138712 |  0.0118125
74 :  -64091.0 4 138714 |  45.0
70 :  -76697.0 96 138738 |  0.0188873626374
45 :  -40112.0 3 138849 |  56.0
71 :  -88001.0 50 138867 |  0.143913043478
74 :  -111517.0 60 139121 |  0.0739393939394
181 :  -40273.0 4 139183 |  64.25
94 :  -140153.0 229 139192 |  0.00430884569309
70 :  -41107.0 3 139241 |  57.3333333333
95 :  -70413.0 77 139253 |  0.0370562770563
28 :  -96296.0 145 139316 |  0.0227180527383
132 :  -60174.0 4 139383 |  45.75
74 :  -53801.0 38 139438 |  0.130827067669
94 :  -47942.0 3 139575 |  54.6666666667
70 :  -106623.0 7 13973

105 :  -45659.0 4 154689 |  45.5
127 :  -40243.0 4 155288 |  54.5
74 :  -69690.0 79 155327 |  0.0464690206529
96 :  -107832.0 55 155339 |  0.0844696969697
95 :  -40158.0 4 155397 |  46.25
103 :  -84808.0 96 155454 |  0.0186170212766
180 :  -88640.0 130 155455 |  0.00979567307692
127 :  -39372.0 4 155496 |  47.25
105 :  -40094.0 4 155508 |  46.25
140 :  -41088.0 4 155553 |  70.5
11 :  -40067.0 3 155558 |  62.3333333333
104 :  -52136.0 28 155696 |  1.10952380952
13 :  -39789.0 3 155722 |  55.0
96 :  -45824.0 6 155723 |  34.6666666667
60 :  -40871.0 4 155727 |  54.75
74 :  -51246.0 12 155885 |  1.97916666667
42 :  -53429.0 3 155895 |  53.0
96 :  -47838.0 23 155896 |  0.889328063241
82 :  -78223.0 100 156391 |  0.0252083333333
13 :  -63847.0 13 156420 |  1.72307692308
50 :  -39500.0 5 156755 |  49.4
95 :  -83853.0 84 156766 |  0.042194092827
96 :  -68829.0 114 156756 |  0.0211000474158
74 :  -44124.0 12 156826 |  1.82291666667
140 :  -40113.0 3 157157 |  62.6666666667
122 :  -44085.0 3 157

158 :  -58930.0 4 190240 |  44.25
72 :  -49085.0 29 190241 |  0.195402298851
74 :  -50065.0 16 190255 |  0.923076923077
95 :  -78128.0 100 190864 |  0.0209278350515
70 :  -63974.0 26 190903 |  0.476588628763
96 :  -59307.0 52 190941 |  0.0906593406593
74 :  -40268.0 4 191339 |  43.75
95 :  -59144.0 29 191348 |  0.241379310345
105 :  -39948.0 3 191903 |  57.0
50 :  -41149.0 6 192042 |  11.5
104 :  -48819.0 3 192181 |  64.3333333333
57 :  -51532.0 33 192700 |  0.180808080808
76 :  -55975.0 45 192852 |  0.188888888889
96 :  -59770.0 29 192921 |  0.277188328912
104 :  -46402.0 20 193061 |  0.491666666667
82 :  -40307.0 3 194123 |  61.0
82 :  -51395.0 18 194372 |  1.13888888889
95 :  -41259.0 4 194529 |  60.25
50 :  -44610.0 6 194644 |  32.5
96 :  -40592.0 7 195329 |  15.2857142857
95 :  -43652.0 10 195342 |  2.91428571429
74 :  -90848.0 4 195349 |  45.0
96 :  -53622.0 7 196806 |  33.4285714286
95 :  -45786.0 16 196816 |  0.869791666667
105 :  -40165.0 3 197102 |  60.6666666667
96 :  -39583

195 :  -602581.0 584 134045 |  0.0102224309342
237 :  -792290.0 586 134045 |  0.00784601597803
169 :  -1134707.0 574 134047 |  0.0155008710801
233 :  -746503.0 577 134048 |  0.0107684829015
18 :  -460541.0 520 134048 |  0.00535684228244
221 :  -1171748.0 644 134048 |  0.0168291751969
77 :  -915636.0 584 134052 |  0.0178848906154
211 :  -1030579.0 578 134053 |  0.0258826286549
226 :  -989643.0 605 134054 |  0.0151121605667
14 :  -762370.0 549 134055 |  0.00831641946396
219 :  -1212326.0 633 134056 |  0.0174921883176
129 :  -794306.0 562 134057 |  0.0130069468921
90 :  -418368.0 479 134055 |  0.00590490998506
143 :  -845576.0 565 134057 |  0.0140417105019
209 :  -690029.0 548 134058 |  0.0173894375268
173 :  -832178.0 572 134059 |  0.0108935182465
128 :  -732025.0 569 134060 |  0.00848910901635
200 :  -698801.0 558 134062 |  0.013083690503
229 :  -1218787.0 598 134077 |  0.0260906183629
131 :  -789208.0 570 134063 |  0.0137528363495
114 :  -1019172.0 566 134063 |  0.0190356776473
242 :  

97 :  -47850.0 29 145101 |  0.288275862069
223 :  -39375.0 5 145139 |  51.0
97 :  -80402.0 27 145232 |  0.308641975309
221 :  -56097.0 4 145255 |  44.25
218 :  -45765.0 4 145263 |  50.5
242 :  -73870.0 92 145334 |  0.0291890571568
156 :  -40134.0 3 145352 |  57.0
229 :  -40006.0 4 145473 |  68.25
75 :  -111014.0 36 145483 |  0.287326388889
218 :  -45944.0 18 145484 |  1.14814814815
47 :  -40005.0 4 146008 |  55.5
154 :  -40123.0 3 146177 |  53.6666666667
210 :  -40343.0 4 146191 |  55.25
246 :  -63266.0 3 146206 |  55.0
175 :  -39971.0 3 146213 |  65.0
75 :  -60618.0 56 146255 |  0.116459627329
243 :  -89115.0 140 146307 |  0.0137643378519
219 :  -40029.0 4 146472 |  61.5
136 :  -40126.0 4 146503 |  50.75
245 :  -48630.0 26 146833 |  0.3127090301
128 :  -40069.0 4 146838 |  44.0
224 :  -40026.0 4 146920 |  46.5
231 :  -40067.0 4 146927 |  57.0
47 :  -63169.0 64 147135 |  0.0455942622951
201 :  -66165.0 75 147230 |  0.03
224 :  -40009.0 4 147234 |  52.5
175 :  -51164.0 31 147457 |  0.23

75 :  -40070.0 4 166200 |  48.5
199 :  -40038.0 4 166285 |  40.75
75 :  -49887.0 8 166405 |  5.95
194 :  -39912.0 5 166618 |  39.4
212 :  -70097.0 81 166625 |  0.0371953149731
156 :  -40165.0 4 166640 |  49.75
221 :  -52853.0 36 166766 |  0.178921568627
75 :  -42283.0 10 167211 |  20.8
138 :  -41741.0 8 167694 |  4.8
97 :  -50972.0 39 167794 |  0.116809116809
75 :  -51978.0 13 167801 |  1.30769230769
97 :  -41520.0 13 167980 |  1.32307692308
185 :  -43743.0 14 167998 |  2.06349206349
169 :  -39995.0 4 169194 |  43.25
172 :  -39989.0 4 169207 |  53.0
97 :  -40025.0 4 169487 |  42.0
75 :  -49383.0 4 169505 |  56.0
147 :  -40060.0 4 169509 |  47.75
113 :  -130496.0 138 169637 |  0.0084450256075
123 :  -54728.0 44 169728 |  0.113053613054
231 :  -48767.0 26 169759 |  0.382943143813
218 :  -40307.0 4 169803 |  68.5
64 :  -40052.0 4 169862 |  49.75
241 :  -39734.0 4 170045 |  57.5
136 :  -40076.0 3 170268 |  57.6666666667
113 :  -83500.0 24 170445 |  0.316287878788
136 :  -57924.0 50 170483 

54 :  -536400.0 497 134068 |  0.00723320090515
41 :  -600535.0 519 134069 |  0.00793855825018
216 :  -865133.0 580 134069 |  0.0159992050084
80 :  -431974.0 514 134072 |  0.00867298143563
67 :  -396563.0 462 134074 |  0.0040014040014
3 :  -401940.0 451 134075 |  0.00572346072767
39 :  -454703.0 485 134077 |  0.00626100075434
254 :  -1559629.0 643 134048 |  0.0257950340537
6 :  -645913.0 478 134089 |  0.0169792289301
106 :  -496979.0 490 134089 |  0.00806760204082
119 :  -778301.0 713 134090 |  0.00188413857587
4 :  -443811.0 439 134091 |  0.008151745677
40 :  -507973.0 458 134091 |  0.00608403162988
101 :  -499680.0 460 134092 |  0.00792039641944
0 :  -346018.0 336 134095 |  0.0049568607931
53 :  -422645.0 469 134096 |  0.00610000480947
35 :  -355600.0 437 134098 |  0.00497999769521
55 :  -352062.0 470 134099 |  0.00337561374795
89 :  -384857.0 451 134099 |  0.00478964682695
56 :  -475899.0 467 134103 |  0.0136127256042
38 :  -537020.0 439 134108 |  0.013010912057
63 :  -435386.0 491 1

120 :  -82836.0 4 175644 |  39.75
126 :  -51560.0 12 175732 |  2.29166666667
22 :  -52914.0 34 175744 |  0.244781783681
106 :  -42986.0 4 175748 |  48.75
121 :  -68146.0 77 176430 |  0.0279045279045
121 :  -40944.0 5 176704 |  43.6
54 :  -40010.0 3 176796 |  55.3333333333
81 :  -54889.0 45 176893 |  0.121693121693
22 :  -40391.0 4 177348 |  63.5
106 :  -49639.0 20 177905 |  0.673076923077
126 :  -46914.0 12 181349 |  2.79761904762
108 :  -40002.0 3 181778 |  64.6666666667
22 :  -43181.0 4 181882 |  46.25
106 :  -96346.0 95 182004 |  0.0274599542334
108 :  -45228.0 5 182016 |  22.8
54 :  -63674.0 20 182071 |  0.570588235294
216 :  -39436.0 4 182107 |  45.75
92 :  -43069.0 12 182198 |  1.47222222222
120 :  -39770.0 3 182211 |  54.3333333333
92 :  -48099.0 5 182463 |  55.8
101 :  -41107.0 4 182922 |  59.75
56 :  -72018.0 35 182926 |  0.387755102041
81 :  -45374.0 7 183438 |  24.1428571429
89 :  -41380.0 7 183612 |  6.21428571429
32 :  -41066.0 4 183631 |  63.25
32 :  -40073.0 3 183864 |  

In [5]:
# A test for channel recording in single event
event = NG.get_single_event(104)
start_time = time.time()

s1_checker = 0
s2_checker = 0

s1_channels = []
s2_channels = []
for p in event.pulses:
    (ch, q, width, t0, sw) = Peaks(p, False)
    if q < 0 and sw != 0:
        sw = -sw
        print(ch, ": ", q, width, t0, "| ", sw)
        if sw > 2:
            if ch not in s1_channels:
                s1_checker += 1
                s1_channels.append(ch)
        else:
            if ch not in s2_channels:
                s2_checker += 1
                s2_channels.append(ch)
end_time = time.time()
long_time = end_time - start_time
print(long_time)
        


153 :  -92581.0 27 100050 |  5.98941798942
105 :  -90150.0 23 100050 |  1.51304347826
11 :  -45133.0 19 100050 |  15.3684210526
112 :  -84573.0 11 100050 |  20.9090909091
164 :  -50831.0 23 100051 |  5.46956521739
165 :  -55266.0 27 100051 |  3.33862433862
13 :  -46121.0 17 100051 |  2.47058823529
95 :  -49849.0 22 100052 |  3.14772727273
141 :  -54073.0 28 100051 |  8.44047619048
151 :  -54081.0 27 100051 |  2.90476190476
76 :  -47396.0 21 100051 |  7.06349206349
204 :  -56210.0 25 100051 |  12.184
180 :  -61451.0 32 100051 |  7.30729166667
181 :  -57360.0 26 100051 |  15.8974358974
189 :  -60486.0 23 100051 |  5.72173913043
177 :  -55290.0 27 100051 |  7.85185185185
42 :  -48929.0 20 100051 |  21.45
71 :  -47242.0 19 100051 |  5.60526315789
96 :  -50425.0 26 100051 |  23.2692307692
110 :  -48204.0 22 100051 |  3.73636363636
122 :  -49999.0 23 100051 |  7.65217391304
46 :  -49423.0 20 100051 |  1.94285714286
159 :  -51959.0 25 100052 |  5.424
149 :  -79572.0 22 100052 |  5.88636363636

105 :  -45659.0 4 154689 |  45.5
127 :  -40243.0 4 155288 |  54.5
74 :  -69690.0 79 155327 |  0.0464690206529
96 :  -107832.0 55 155339 |  0.0844696969697
95 :  -40158.0 4 155397 |  46.25
103 :  -84808.0 96 155454 |  0.0186170212766
180 :  -88640.0 130 155455 |  0.00979567307692
127 :  -39372.0 4 155496 |  47.25
105 :  -40094.0 4 155508 |  46.25
140 :  -41088.0 4 155553 |  70.5
11 :  -40067.0 3 155558 |  62.3333333333
104 :  -52136.0 28 155696 |  1.10952380952
13 :  -39789.0 3 155722 |  55.0
96 :  -45824.0 6 155723 |  34.6666666667
60 :  -40871.0 4 155727 |  54.75
74 :  -51246.0 12 155885 |  1.97916666667
42 :  -53429.0 3 155895 |  53.0
96 :  -47838.0 23 155896 |  0.889328063241
82 :  -78223.0 100 156391 |  0.0252083333333
13 :  -63847.0 13 156420 |  1.72307692308
50 :  -39500.0 5 156755 |  49.4
95 :  -83853.0 84 156766 |  0.042194092827
96 :  -68829.0 114 156756 |  0.0211000474158
74 :  -44124.0 12 156826 |  1.82291666667
140 :  -40113.0 3 157157 |  62.6666666667
122 :  -44085.0 3 157

KeyboardInterrupt: 

In [11]:
print('s1_channels: ' +  str(s1_checker))
s1_channels.sort()
print(s1_channels)
print('s2_channels: ' +  str(s2_checker))
s2_channels.sort()
print(s2_channels)

s1_channels: 192
[4, 6, 9, 10, 11, 13, 16, 17, 19, 21, 22, 23, 24, 28, 30, 32, 33, 35, 38, 39, 40, 41, 42, 44, 45, 46, 47, 48, 49, 50, 51, 53, 54, 55, 56, 57, 60, 63, 64, 66, 67, 68, 69, 70, 71, 72, 74, 75, 76, 77, 78, 80, 81, 82, 83, 84, 85, 89, 90, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 103, 104, 105, 106, 107, 108, 110, 111, 112, 113, 114, 115, 116, 117, 119, 120, 121, 122, 123, 125, 126, 127, 128, 129, 131, 132, 133, 136, 138, 140, 141, 142, 143, 144, 145, 146, 147, 149, 151, 153, 154, 155, 156, 157, 158, 159, 160, 161, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 179, 180, 181, 182, 184, 185, 186, 187, 188, 189, 191, 192, 193, 194, 195, 196, 197, 199, 200, 201, 202, 204, 205, 207, 208, 209, 210, 211, 212, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 235, 236, 237, 238, 240, 241, 242, 243, 245, 246, 247, 254]
s2_channels: 216
[0, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23

In [11]:
# Test: how long does it take if we record all channels by ourselves


event = NG.get_single_event(104)


s1_channellist = []
s2_channellist = []

s1_checker = 0
s2_checker = 0

s1_q = 0
s1_t0 = 0
s2_q = 0
s2_t0 = 0

start_time = time.time()

for p in event.pulses:
    (ch, q, width, t0, sw) = Peaks(p, False)
    
    if q < 0 and sw != 0:
        sw = -sw
        if sw > 2:
            if ch in det_ch['tpc'] and ch not in s1_channellist:
                s1_channellist.append(ch)
                s1_checker += 1
            if ch == 254:
                s1_q = -q
                s1_t0 = t0
        else:
            if s2_q == 0:
                s2_t0 = t0
            if ch in det_ch['tpc'] and ch not in s2_channellist:
                s2_channellist.append(ch)
                s2_checker += 1
            s2_q = s2_q - q
if s1_checker>2 and s2_checker>3:
    info = {'event': event.event_number, 's1': s1_q, 's2': s2_q, 
            's1_position': s1_t0, 's2_position': s2_t0}

end_time = time.time()
print(s1_checker, s2_checker)
print(end_time-start_time)

186 215
52.59764790534973


## The main function: s1s2\_info

In [3]:
# Discriminate S1 and S2 peaks by the condition: Slope/Width = 2
# Record peaks found in Channel 254
# Peaks in other channels are used as criteria:
# Only S1 with at least 3 PMT's response / S2 with at least 4 PMT's response can be recorded
# s1, s2: the charge of these two peaks
# s1_position, s2_position: the time point where the first S1 and S2 appear


def s1s2_info(file):
    info_list = []
    #width_list = []
    #sw_list = []

    start_time = time.time()

    for event in file.get_events():
        if event.event_number%100 == 0:
            print(event.event_number, time.time()-start_time)
    
        s1_q = 0
        s1_t0 = 0
        s2_q = 0
        s2_t0 = 0
    
        s1_channellist = []
        s2_channellist = []
    
        s1_checker = 0
        s2_checker = 0
    
        for p in event.pulses:
            if p.channel == 254:
                (ch, q, width, t0, sw) = Peaks(p, False)
                if q < 0 and sw != 0:
                    sw = -sw
                    if sw > 2:
                        if s1_q == 0:
                            s1_q = -q
                            s1_t0 = t0
                    else:
                        if s2_q == 0:
                            s2_t0 = t0
                        s2_q = s2_q - q
            else:
                if s1_checker<3 or s2_checker<4:
                    (ch, q, width, t0, sw) = Peaks(p, False)
                    if q<0 and sw!=0:
                        sw = -sw
                        if sw > 2 and ch not in s1_channellist:
                            s1_channellist.append(ch)
                            s1_checker += 1
                        if sw < 2 and ch not in s2_channellist:
                            s2_channellist.append(ch)
                            s2_checker += 1
                    
        if s1_q and s2_q and s1_checker>2 and s2_checker>3:
            info = {'event': event.event_number, 's1': s1_q, 's2': s2_q,
                    's1_position': s1_t0, 's2_position': s2_t0}
            info_list.append(info)
        if event.event_number == 10000:
            break

    
    return(info_list)
    


In [4]:
info_NG = s1s2_info(NG)

0 0.09939122200012207
100 70.77067255973816
200 1702.1590230464935
300 1915.709181547165
400 2002.3010077476501
500 2229.730314731598
600 2309.6565272808075
700 2398.685202360153
800 2475.9856431484222
900 2563.931358575821
1000 2645.767874479294
1100 2724.0170335769653
1200 2812.127944946289
1300 2894.473902463913
1400 3212.6571741104126
1500 3304.702585697174
1600 3402.7751808166504
1700 3513.5138607025146
1800 3612.7248363494873
1900 3688.0026047229767
2000 3769.936101913452
2100 3878.61225438118
2200 3967.16294670105
2300 4101.569822311401
2400 4205.587690591812
2500 4294.163521051407
2600 4378.418070077896
2700 4473.8311586380005
2800 4564.535282611847
2900 4705.814389705658
3000 4811.321962356567
3100 5086.748854160309
3200 5173.803074359894
3300 5264.76162147522
3400 5365.623729705811
3500 5473.128546953201
3600 5629.413297653198
3700 5723.700984954834
3800 5802.753534078598
3900 5882.661197185516
4000 6029.66033577919
4100 6105.018622875214
4200 6232.6587953567505
4300 6328.324

In [5]:
print(len(info_NG))

7490


In [6]:
#See structure of the list

print(info_NG[300])

{'event': 460, 's1': 48618.0, 's2': 213835.0, 's1_position': 100055, 's2_position': 139300}


## Save NG, run for Rn220

In [7]:
with open('NG_data_new.p', 'wb') as fp:
    pickle.dump(info_NG, fp, protocol=pickle.HIGHEST_PROTOCOL)

In [8]:
with open('NG_data.p', 'rb') as fp:
    data = pickle.load(fp)

In [9]:
print(data[300])

{'event': 460, 's1': 48618.0, 's2': 213835.0, 's1_position': 100055, 's2_position': 139300}


In [10]:
info_Rn220 = s1s2_info(Rn220)

0 0.0049190521240234375
100 86.87878203392029
200 161.79596853256226
300 258.3929717540741
400 321.53019189834595
500 391.2320249080658
600 466.9542806148529
700 549.8134655952454
800 627.20654296875
900 710.2618362903595
1000 777.0041999816895
1100 848.367838382721
1200 924.5054843425751
1300 988.2122783660889
1400 1067.9878780841827
1500 1151.0677525997162
1600 1226.545264005661
1700 1311.5586695671082
1800 1376.0305049419403
1900 1463.9967131614685
2000 1575.7737836837769
2100 1654.5254652500153
2200 2760.969980239868
2300 2834.965362071991
2400 2910.0272932052612
2500 2989.757192134857
2600 3060.4156765937805
2700 3147.470681667328
2800 3228.0317330360413
2900 3300.404347896576
3000 3383.334814786911
3100 3458.401534318924
3200 3541.163832426071
3300 3622.116421699524
3400 3697.5482382774353
3500 3768.6619279384613
3600 3844.1108815670013
3700 3925.881514072418
3800 4002.8061015605927
3900 4069.844987630844
4000 4143.762972354889
4100 4224.523337841034
4200 4298.668357133865
4300 4

In [11]:
print(len(info_Rn220))

5921


In [12]:
with open('Rn220_data_new.p', 'wb') as fp:
    pickle.dump(info_Rn220, fp, protocol=pickle.HIGHEST_PROTOCOL)

In [13]:
print(info_Rn220[200])

{'event': 343, 's1': 58677.0, 's2': 361574.0, 's1_position': 100053, 's2_position': 169954}
